In [3]:
# prompt
import openai
import os
import json

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.getenv('OPENAI_API_KEY')

In [4]:
# read prompt
with open('prompt_with_examples.txt') as f:
    system_prompt = f.read()

system_prompt

'You are an experienced data labeling engineer with extensive experience in labeling affiliation informations. Your task is to use your comprehensive knowledge of business and management schools to complete seven attributes. If any one attribute is not specified in the provided information please deduce by other attributes. \n\nI will provide you with several examples:\n1. Deduce "university/company/institute" by "department" and "college/school/faculty" \nexample input: Professor of information systems and management, Heinz College expected output: {\n  "address": "5000 Forbes Avenue, Pittsburgh, PA 15213, USA",\n  "department": "Information Systems and Management",\n  "college/school/faculty": "Heinz College of Information Systems and Public Policy",\n  "if it\'s a business/management school": false,\n  "university/company/institute": "Carnegie Mellon University",\n  "country": "US",\n  "city": "Pittsburgh"\n}\n2. deduce "college/school/faculty" by other attributes\nexample input: De

In [5]:
import json

with open('/Users/yihong_eric_chen/Desktop/buzpub/papers_info.json') as f:
    paper_data = json.load(f)


In [6]:
# test 随机选取十篇文章
import random
random.seed(0)
test = random.sample(paper_data, 10)
test

[{'title': 'the early impact of gdpr compliance on display advertising: the case of an ad publisher.',
  'abstract': "The European Union's General Data Protection Regulation (GDPR), with its explicit consent requirement, may restrict the use of personal data and shake the foundations of online advertising. The ad industry has predicted drastic loss of revenue from GDPR compliance and has been seeking alternative ways of targeting. Taking advantage of an event created by an ad publisher's request for explicit consent from users with European Union IP addresses, the authors find that for a publisher that uses a pay-per-click model, has the capacity to leverage both user behavior and web page content information for advertising, and observes high consent rates, GDPR compliance leads to modest negative effects on ad performance, bid prices, and ad revenue. The changes in ad metrics can be explained by temporal variations in consent rates. The impact is most pronounced for travel and financ

In [7]:
from openai import OpenAI
client = OpenAI()


In [8]:
model="gpt-3.5-turbo-0125"

total_cost = 0
def deduce_attributes(affiliation,total_cost):
    
    attributes = {
        "address": "",
        "department": "",
        "college/school/faculty": "",
        "if it's a business/management school": None,
        "university/company/institute": "",
        "country": "",
        "city": ""
    }

    response = client.chat.completions.create(
    model=model,
    temperature=0,
    response_format={ "type": "json_object" },
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": affiliation}
        ]
    )

    # 假设response正确返回了预期的JSON对象格式
    content_dict = json.loads(response.choices[0].message.content)
    # 更新属性
    for key in attributes.keys():
        if key in content_dict:
            attributes[key] = content_dict[key]
    total_cost += response.usage.total_tokens* 0.5/1000000
    return attributes, total_cost


In [9]:

for paper in test:
    for author in paper["authors"]:
        affiliation = author["affiliation"]
        deduced_attributes,total_cost = deduce_attributes(affiliation,total_cost)
        author.update(deduced_attributes)


In [10]:
print(f"Total cost: ${total_cost}")

Total cost: $0.0060805


In [11]:
print(test)

[{'title': 'the early impact of gdpr compliance on display advertising: the case of an ad publisher.', 'abstract': "The European Union's General Data Protection Regulation (GDPR), with its explicit consent requirement, may restrict the use of personal data and shake the foundations of online advertising. The ad industry has predicted drastic loss of revenue from GDPR compliance and has been seeking alternative ways of targeting. Taking advantage of an event created by an ad publisher's request for explicit consent from users with European Union IP addresses, the authors find that for a publisher that uses a pay-per-click model, has the capacity to leverage both user behavior and web page content information for advertising, and observes high consent rates, GDPR compliance leads to modest negative effects on ad performance, bid prices, and ad revenue. The changes in ad metrics can be explained by temporal variations in consent rates. The impact is most pronounced for travel and financia

In [12]:
# test0 to json
import json
with open('test0.json', 'w') as f:
    json.dump(test, f, indent=4)